In [12]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData

from autoencoder_dataset import autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoder
from train_funcs import train_autoencoder_dataloader
from test_funcs import test_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'mpi-mesh' # 'mpi-mesh', 'trimesh'
root_dir = '/home/jingwang/Data/data/'

dataset = 'FaceWarehouse'
name = 'align_pose'

torch.backends.cudnn.benchmark = True


GPU = True
device_idx = 0 # 0, 1, 2, 3
device_ids = [0, 1, 2, 3]
for idx in device_ids:
    print(torch.cuda.get_device_name(idx))

GeForce GTX TITAN X
GeForce GTX TITAN X
GeForce GTX TITAN X
GeForce GTX TITAN X


In [13]:
args = {}

generative_model = 'simple_autoencoder'
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'


# below are the arguments for the DFAUST run
reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')
downsample_directory = os.path.join(root_dir, dataset,'template', downsample_method)
ds_factors = [4]
step_sizes = [1, 1]
filter_sizes_enc = [[3, 32],[[],[]]]
filter_sizes_dec = [[32, 16],[[],3]]
dilation_flag = True
if dilation_flag:
    dilation=[1, 1] 
else:
    dilation = None
reference_points = [[5930]]# [[3567,4051,4597]] # [[414]]  # used for COMA with 3 disconnected components

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/spirals_'+ generative_model),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size': 96, 'num_epochs':300, 'eval_frequency':200, 'num_workers': 40,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz': 128, # 100 identity + 46 expression 
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,
        
        'lr':1e-3, 
        'regularization': 5e-5,         
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': False,
        
        'mode':'train', 'shuffle': True, 'nVal': 100, 'normalization': True,
        'write_mesh': True,
        'lambda_var': 0.5,
        'worst_face_num': 20}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

downsample_mesh_path = os.path.join(args['results_folder'],'downsample_mesh', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)


worst_mesh_path = os.path.join(args['results_folder'],'worst_test', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)

In [14]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]

if not os.path.exists(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl')):
    if shapedata.meshpackage == 'trimesh':
        raise NotImplementedError('Rerun with mpi-mesh as meshpackage')
    print("Generating Transform Matrices ..")
    if downsample_method == 'COMA_downsample':
        M,A,D,U,F = mesh_sampling.generate_transform_matrices(shapedata.reference_mesh, args['ds_factors'])
        if args['write_mesh']:
            import openmesh
            for i in range(len(M)):
                mesh = openmesh.TriMesh(points=M[i].v,face_vertex_indices=M[i].f)
                openmesh.write_mesh(os.path.join(args['results_folder'],'downsample_mesh',args['name'], '%d.obj'%i),mesh)
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'wb') as fp:
        M_verts_faces = [(M[i].v, M[i].f) for i in range(len(M))]
        pickle.dump({'M_verts_faces':M_verts_faces,'A':A,'D':D,'U':U,'F':F}, fp)
else:
    print("Loading Transform Matrices ..")
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'rb') as fp:
        #downsampling_matrices = pickle.load(fp,encoding = 'latin1')
        downsampling_matrices = pickle.load(fp)
            
    M_verts_faces = downsampling_matrices['M_verts_faces']
    if shapedata.meshpackage == 'mpi-mesh':
        from psbody.mesh import Mesh
        M = [Mesh(v=M_verts_faces[i][0], f=M_verts_faces[i][1]) for i in range(len(M_verts_faces))]
    elif shapedata.meshpackage == 'trimesh':
        M = [trimesh.base.Trimesh(vertices=M_verts_faces[i][0], faces=M_verts_faces[i][1], process = False) for i in range(len(M_verts_faces))]
    A = downsampling_matrices['A']
    D = downsampling_matrices['D']
    U = downsampling_matrices['U']
    F = downsampling_matrices['F']
        
# Needs also an extra check to enforce points to belong to different disconnected component at each hierarchy level
print("Calculating reference points for downsampled versions..")
for i in range(len(args['ds_factors'])):
    if shapedata.meshpackage == 'mpi-mesh':
        dist = euclidean_distances(M[i+1].v, M[0].v[reference_points[0]])
    elif shapedata.meshpackage == 'trimesh':
        dist = euclidean_distances(M[i+1].vertices, M[0].vertices[reference_points[0]])
    reference_points.append(np.argmin(dist,axis=0).tolist())



Loading data .. 
Loading Transform Matrices ..
Calculating reference points for downsampled versions..


In [15]:
if shapedata.meshpackage == 'mpi-mesh':
    sizes = [x.v.shape[0] for x in M]
elif shapedata.meshpackage == 'trimesh':
    sizes = [x.vertices.shape[0] for x in M]
Adj, Trigs = get_adj_trigs(A, F, shapedata.reference_mesh, meshpackage = shapedata.meshpackage)

spirals_np, spiral_sizes,spirals = generate_spirals(args['step_sizes'], 
                                                    M, Adj, Trigs, 
                                                    reference_points = reference_points, 
                                                    dilation = args['dilation'], random = False, 
                                                    meshpackage = shapedata.meshpackage, 
                                                    counter_clockwise = True)

bU = []
bD = []
for i in range(len(D)):
    d = np.zeros((1,D[i].shape[0]+1,D[i].shape[1]+1))
    u = np.zeros((1,U[i].shape[0]+1,U[i].shape[1]+1))
    d[0,:-1,:-1] = D[i].todense()
    u[0,:-1,:-1] = U[i].todense()
    d[0,-1,-1] = 1
    u[0,-1,-1] = 1
    bD.append(d)
    bU.append(u)


spiral generation for hierarchy 0 (11510 vertices) finished
spiral generation for hierarchy 1 (2878 vertices) finished
spiral sizes for hierarchy 0:  10
spiral sizes for hierarchy 1:  9


In [16]:
torch.manual_seed(args['seed'])

if GPU:
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

tspirals = [torch.from_numpy(s).long().to(device) for s in spirals_np]
tD = [torch.from_numpy(s).float().to(device) for s in bD]
tU = [torch.from_numpy(s).float().to(device) for s in bU]

cuda:0


In [17]:
tspirals[0].shape

torch.Size([1, 11511, 10])

In [18]:
# Building model, optimizer, and loss function

dataset_train = autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                           shapedata = shapedata,
                                           normalization = args['normalization'])

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'], num_workers = args['num_workers'], pin_memory=True)

dataset_val = autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                         shapedata = shapedata,
                                         normalization = args['normalization'])

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'], pin_memory=True)


dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'],  pin_memory=True)



if 'autoencoder' in args['generative_model']:
        model = torch.nn.DataParallel(SpiralAutoencoder(filters_enc = args['filter_sizes_enc'],   
                                  filters_dec = args['filter_sizes_dec'],
                                  latent_size=args['nz'],
                                  sizes=sizes,
                                  spiral_sizes=spiral_sizes,
                                  spirals=tspirals,
                                  D=tD, U=tU).to(device),device_ids=device_ids)
        if torch.cuda.device_count() > 1:
#             model = torch.nn.parallel.DataParallel(model, device_ids=device_ids)
            print('Let\'s use %d GPUs!'%torch.cuda.device_count())
 
    
optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

def loss_l2(outputs, targets):
    L = torch.sqrt(torch.mean((outputs - targets)**2))
    return L
    
if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1
elif arg['loss']=='l1_var':
    lambda_var = args['lambda_var']
    def variational_loss(tx,tx_hat):
        x,mu,logvar = tx
        l1_loss = torch.mean(torch.abs(x-tx))
        var_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return l1_loss + lambda_var * var_loss
    loss_fn = variational_loss



Let's use 4 GPUs!


/home/jingwang/Data/anaconda3/envs/py2-spiral/lib/python2.7/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [19]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 23683635
DataParallel(
  (module): SpiralAutoencoder(
    (conv): ModuleList(
      (0): SpiralConv(
        (conv): Linear(in_features=30, out_features=32, bias=True)
        (activation): ELU(alpha=1.0)
      )
    )
    (fc_latent_enc): Linear(in_features=92128, out_features=128, bias=True)
    (fc_latent_dec): Linear(in_features=128, out_features=92128, bias=True)
    (dconv): ModuleList(
      (0): SpiralConv(
        (conv): Linear(in_features=320, out_features=16, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): SpiralConv(
        (conv): Linear(in_features=160, out_features=3, bias=True)
      )
    )
  )
)


In [10]:
128*5023

642944

In [11]:
2688/128

21

In [14]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    if args['generative_model'] == 'autoencoder':
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn,
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=True,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])

100%|██████████| 2/2 [00:05<00:00,  3.92s/it]


epoch 0 | tr 0.513800279745 | val 0.335121604204


100%|██████████| 2/2 [00:05<00:00,  2.94s/it]


epoch 1 | tr 0.276958546967 | val 0.25284075439


100%|██████████| 2/2 [00:05<00:00,  2.99s/it]


...

epoch 195 | tr 0.0440829874883 | val 0.0803302931786


100%|██████████| 2/2 [00:05<00:00,  3.56s/it]


epoch 196 | tr 0.0436791906454 | val 0.0794854050875


  0%|          | 0/97 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
model.module.only_decode

False

In [ ]:
model.module.fc_latent_dec.weight.device

In [32]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar


100%|██████████| 7/7 [00:07<00:00,  1.02it/s]


('loss for ', 402, tensor(2.7376, device='cuda:0'))
('loss for ', 560, tensor(2.7437, device='cuda:0'))
('loss for ', 50, tensor(2.7965, device='cuda:0'))
('loss for ', 537, tensor(2.7487, device='cuda:0'))
('loss for ', 390, tensor(2.8354, device='cuda:0'))
('loss for ', 364, tensor(2.8189, device='cuda:0'))
('loss for ', 169, tensor(2.8539, device='cuda:0'))
('loss for ', 616, tensor(2.7999, device='cuda:0'))
('loss for ', 341, tensor(2.7889, device='cuda:0'))
('loss for ', 590, tensor(2.8523, device='cuda:0'))
('loss for ', 499, tensor(2.9018, device='cuda:0'))
('loss for ', 389, tensor(2.8896, device='cuda:0'))
('loss for ', 125, tensor(2.8902, device='cuda:0'))
('loss for ', 370, tensor(2.9148, device='cuda:0'))
('loss for ', 614, tensor(2.8942, device='cuda:0'))
('loss for ', 600, tensor(2.9359, device='cuda:0'))
('loss for ', 194, tensor(2.8669, device='cuda:0'))
('loss for ', 8, tensor(2.8613, device='cuda:0'))
('loss for ', 653, tensor(2.8374, device='cuda:0'))
('loss for ', 5

In [18]:
if args['mode'] == 'test': # test with train set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_train, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar



100%|██████████| 97/97 [00:36<00:00,  3.36it/s]

('loss for ', 586, tensor(0.6500, device='cuda:0'))
('loss for ', 2114, tensor(0.6520, device='cuda:0'))
('loss for ', 7367, tensor(0.6512, device='cuda:0'))
('loss for ', 2851, tensor(0.6557, device='cuda:0'))
('loss for ', 6893, tensor(0.6568, device='cuda:0'))
('loss for ', 8053, tensor(0.6522, device='cuda:0'))
('loss for ', 6181, tensor(0.6723, device='cuda:0'))
('loss for ', 2481, tensor(0.6690, device='cuda:0'))
('loss for ', 6458, tensor(0.6588, device='cuda:0'))
('loss for ', 6215, tensor(0.6871, device='cuda:0'))
('loss for ', 4388, tensor(0.7073, device='cuda:0'))
('loss for ', 3775, tensor(0.8443, device='cuda:0'))
('loss for ', 3728, tensor(0.6557, device='cuda:0'))
('loss for ', 645, tensor(0.7630, device='cuda:0'))
('loss for ', 4929, tensor(0.7139, device='cuda:0'))
('loss for ', 1636, tensor(0.6774, device='cuda:0'))
('loss for ', 2921, tensor(0.7664, device='cuda:0'))
('loss for ', 8807, tensor(0.6903, device='cuda:0'))
('loss for ', 2249, tensor(0.6622, device='cuda:

In [39]:
# from optimize_to_get_result import optimize_to_get_result

import torch
import torch.optim as optim
import torch.nn as nn


def optimize_to_get_result(model, loss_fun, z_dim, device, targets, n_iter=1, output_loss=True):
    model.eval()
    
#     return torch.empty(0), model(targets)
    
    model.module.only_encode = True
    z = model(targets)
    model.module.only_encode = False
    
    model.module.only_decode = True
    z_param = nn.Parameter(z)
    optimizer = optim.LBFGS(params=[z_param])
    for it in range(n_iter):
        def closure():
            optimizer.zero_grad()
            outputs = model(z_param)
            loss = loss_fun(outputs, targets)
            loss.backward()
            if output_loss:
                print('loss ', loss.item())
            return loss
        optimizer.step(closure)
    outputs = model(z_param)
    
    model.module.only_decode = False
    return z_param.data, outputs

def work_optimize():

    test_dataset = dataset_test
    # test_ids = [402,560, 50,  537, 390,
    #             364, 169, 616, 341, 590,
    #             499, 389,125, 370, 614,
    #             600, 194, 8, 653, 559]
    test_ids = [8]
    inputs = torch.cat([test_dataset[idx]['points'].to(device).unsqueeze(0) for idx in test_ids])
    inputs = inputs.to(device)

    print(inputs.shape) # fixme

    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    z, outputs = optimize_to_get_result(model, loss_fn, args['nz'], device,
                          inputs)
    from pprint import pprint
    pprint(z)

    outputs = outputs[:,:-1]
    inputs = inputs[:,:-1]

    old_outputs = outputs
    old_inputs = inputs

    outputs = outputs * shapedata_std + shapedata_mean
    inputs = inputs * shapedata_std + shapedata_mean

    per_l2_loss = torch.sqrt(torch.sum((outputs - inputs)**2, dim=2))

    for i in range(len(test_ids)):
        print(test_ids[i], torch.mean(per_l2_loss[i]))

    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_input'),old_inputs.detach().cpu().numpy(),test_ids)
    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_output'),old_outputs.detach().cpu().numpy(),test_ids)

work_optimize()

torch.Size([1, 11511, 3])
('loss ', 0.39739155769348145)
('loss ', 0.39277511835098267)
('loss ', 0.35588905215263367)
('loss ', 0.33115866780281067)
('loss ', 0.3164112865924835)
('loss ', 0.3067980408668518)
('loss ', 0.2976946234703064)
('loss ', 0.2918466627597809)
('loss ', 0.2884432375431061)
('loss ', 0.28520214557647705)
('loss ', 0.2826074957847595)
('loss ', 0.27986764907836914)
('loss ', 0.27789127826690674)
('loss ', 0.276755154132843)
('loss ', 0.275785356760025)
('loss ', 0.27493956685066223)
('loss ', 0.2742707133293152)
('loss ', 0.27380314469337463)
('loss ', 0.2734643816947937)
('loss ', 0.2732097804546356)
tensor([[-1.2232,  0.2258,  0.2435, -0.7786,  2.4340, -0.1475, -0.0689,  0.4264,
         -0.0836, -0.2713,  0.8224,  0.8065,  0.2474,  0.4163, -0.6270, -0.9571,
          4.9992, -0.8653,  1.1199, -0.8996, -3.6033, -0.4942,  0.4912, -0.3746,
         -1.2572,  1.9690, -0.7755, -0.2962,  0.1613, -1.5031,  1.9733,  0.6295,
         -1.8684, -1.2721,  1.0964,  0.7270

In [19]:
type(inputs)

torch.Tensor

In [20]:
type(outputs)

torch.Tensor

In [21]:
type(old_inputs)

torch.Tensor

In [29]:
torch.cat([test_dataset[0]['points'].unsqueeze(0),test_dataset[1]['points'].unsqueeze(0)]).shape

torch.Size([2, 11511, 3])

In [21]:
args['nz']

128

In [23]:
model.only_decode

True

In [27]:
z1 = torch.zeros((2,11511,3))
z2 = torch.zeros((2,128))

model(z1)

tensor([[[-0.0581,  0.0713, -0.0968],
         [-0.0582,  0.0715, -0.0978],
         [-0.0570,  0.0703, -0.0970],
         ...,
         [-0.0562,  0.0718, -0.0985],
         [-0.0539,  0.0685, -0.1012],
         [-0.0000,  0.0000, -0.0000]],

        [[-0.0581,  0.0713, -0.0968],
         [-0.0582,  0.0715, -0.0978],
         [-0.0570,  0.0703, -0.0970],
         ...,
         [-0.0562,  0.0718, -0.0985],
         [-0.0539,  0.0685, -0.1012],
         [-0.0000,  0.0000, -0.0000]]], device='cuda:0',
       grad_fn=<GatherBackward>)